In [1]:
import erddapy
from erddapy import ERDDAP
import numpy as np
import pandas as pd
import xarray
import cf_xarray
import datetime
import netCDF4
from netCDF4 import Dataset

import matplotlib
from matplotlib import pyplot as plt

In [2]:
def get_erddap_data(erddap_url, dataset, data_protocol="griddap", variables=None, constraints=None):
    """
    Function: get_erddap_data
    This function uses the erddapy python library to access data from ERDDAP servers,
    and to return it to users in convenient formats for python users.
    Data can be pulled from "tabledap" or "griddap" formats, with different
    output types, depending on the dap type.
    
    Inputs:
    erddap_url    - The url address of the erddap server to pull data from
    variables     - The selected variables within the dataset.
    data_protocol - The erddap data protocol for the chosen dataset.
                    Options include "tabledap" or "griddap"
                    The default option is given as "griddap"
    dataset       - The ID for the relevant dataset on the erddap server
                    If no variables are given, it is assumed that all variables
                    will be pulled.
    constraints   - These are set by the user to help restrict the data pull
                    to only the area and timeframe of interest.
                    If no constraints are given, all data in a dataset is pulled.
                    Constraints should be given as a dictionary, where
                    each entry is a bound and/or selection of a specific axis variable
                    Exs. {"longitude<=": "min(longitude)+10", "longitude>=": "0"}
                         {"longitude=": "140", "time>=": "max(time)-30"}
    
    Outputs:
    erddap_data   - This variable contains the pulled data from the erddap server.
                    If the data_protocol is "griddap",  then erddap_data is an xarray dataset
                    If the data_protocol is "tabledap", then erddap_data is a pandas dataframe
    """
    
    import erddapy
    from erddapy import ERDDAP
    import pandas as pd
    import xarray
    
    
    ############################################
    # Set-up the connection to the ERDDAP server
    ############################################
    
    # Connect to the erddap server
    e = ERDDAP(server=erddap_url, protocol=data_protocol, response='csv')
    
    # Identify the dataset of interest
    e.dataset_id = dataset
    
    
    #########################################
    # Pull the data, based upon protocol type
    #########################################
    
    # GRIDDAP Protocol
    if data_protocol == "griddap":
        
        # Initialize the connection
        e.griddap_initialize()

        # Update the constraints
        if constraints is not None:
            e.constraints.update(constraints)
            e.griddap_initialize()
            
        # Update the selection of the variables
        if variables is not None:
            e.variables = variables

        erddap_data = e.to_xarray()
    
    # TABLEDAP Protocol
    elif data_protocol == "tabledap":

        # Update the constraints
        if constraints is not None:
            e.constraints = constraints
            
        # Update the selection of the variables
        if variables is not None:
            e.variables = variables
            
        erddap_data = e.to_pandas()
    
    # Invalid protocol given
    else:
        print('Invalid ERDDAP protocol. Given protocol is: ' + data_protocol)
        print('Valid protocols include "griddap" or "tabledap". Please restart and try again with a valid protocol')
        erddap_data = None
    
    
    #############################
    return erddap_data

In [ ]:
#check pudget sound mooring data
nwem_url = 'http://nwem.apl.washington.edu/erddap'

nwem_dataset = 'orca3_L3_depthgridded_025'

variables = ["sea_water_temperature",
             "sea_water_practical_salinity"]

constraints = {"cast_start_time>=":datetime.datetime(2022,1,1).strftime('%Y-%m-%dT%H:%M:%SZ')}
#constraints = {"cast_start_time>=": "max(cast_start_time)-365"}

nwem_grid = get_erddap_data(nwem_url, nwem_dataset, 
                variables=variables, 
                constraints=constraints,
                data_protocol="griddap")

In [ ]:
nwem_grid

In [ ]:
nwem_grid.sea_water_temperature.sel(depth=30).plot()

In [ ]:
# show the bounding polygon to search E01 data

import folium

#set the boundary box
min_lat, max_lat = 49.28, 49.30
min_lon, max_lon = -126.62, -126.60

lon = (min_lon + max_lon) / 2
lat = (min_lat + max_lat) / 2

m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=5)

folium.Polygon(
    [(min_lat, min_lon), (max_lat, min_lon), (max_lat, max_lon), (min_lat, max_lon)],
    fill=True,
).add_to(m)


In [ ]:
#check DFO mooring E01 data
cioos_url = 'https://data.cioospacific.ca/erddap'
cioos_dataset = 'IOS_CTD_Moorings'

variables = ["time",
             "depth",
             "latitude",
             "longitude",
             "sea_water_pressure",
             "sea_water_temperature",
             "sea_water_practical_salinity",
             "TEMPS901",
             "TEMPS902",
             "TEMPS601",
             "TEMPS602",
             "TEMPST01"]

#constraints = {"time>=":datetime.datetime(2022,1,1).strftime('%Y-%m-%dT%H:%M:%SZ')}
#constraints = {"time>=": "max(time)-365"}
constraints = {
    "latitude>=": 49.28,
    "latitude<=": 49.30,
    "longitude>=": -126.62,
    "longitude<=": -126.60,
}

cioos_table = get_erddap_data(cioos_url, cioos_dataset, 
                variables=variables,
                constraints=constraints,
                data_protocol="tabledap")

In [ ]:
cioos_table

In [ ]:
cioos_table.columns

In [ ]:
cioos_table_1 = cioos_table.rename(columns={'time (UTC)': 'time',
                                            'depth (m)': 'depth', 
                                            'latitude (degrees_north)': 'latitude',
                                            'longitude (degrees_east)': 'longitude',
                                            'sea_water_pressure (dbar)': 'sea_water_pressure',
                                            'sea_water_temperature (degC)': 'sea_water_temperature',
                                            'sea_water_practical_salinity (PSS-78)': 'sea_water_pratical_salinity',
                                            'TEMPS901 (deg_C)': 'TEMPS901',
                                            'TEMPS902 (deg_C)': 'TEMPS902',
                                            'TEMPS601 (deg_C)': 'TEMPS601',
                                            'TEMPS602 (deg_C)': 'TEMPS602',
                                            'TEMPST01 (deg_C)': 'TEMPST01'})

In [ ]:
cioos_table_1.columns

In [ ]:
#confirm the time is in string
type(cioos_table_1.time[0])

In [ ]:
cioos_table_1 = cioos_table_1.reset_index().set_index('time')
cioos_table_1.sea_water_temperature.plot()

In [ ]:
cioos_table_1.sea_water_pratical_salinity.plot()

In [ ]:
cioos_table_1.TEMPS901.plot()

In [ ]:
cioos_table_1.TEMPST01.plot() 

In [ ]:
# confirm the na values for TEMPST01
cioos_table_1.TEMPST01.dropna()

In [ ]:
cioos_table_1.TEMPS902.plot()

In [ ]:
# confirm the na values for TEMPS902
cioos_table_1.TEMPS902.dropna()

In [ ]:
cioos_table_1.TEMPS601.plot()

In [ ]:
# confirm the na values for TEMPS601
cioos_table_1.TEMPS601.dropna()

In [ ]:
cioos_table_1.TEMPS602.plot()

In [ ]:
# confirm the na values for TEMPS602
cioos_table_1.TEMPS602.dropna()

In [ ]:
# could drop columns without real data
cioos_table_1 = cioos_table_1.drop(['TEMPS901', 'TEMPS902', 'TEMPS601', 'TEMPS602'], axis=1)

In [ ]:
cioos_table_1.columns

In [ ]:
cioos_table_1.depth.plot()

In [ ]:
cioos_table_1[(cioos_table_1["depth"] >= 30) & (cioos_table_1["depth"] <= 40)].TEMPST01.plot()

In [ ]:
cioos_e01_35 = cioos_table_1[(cioos_table_1["depth"] >= 30) & (cioos_table_1["depth"] <= 40)]
cioos_e01_75 = cioos_table_1[(cioos_table_1["depth"] >= 70) & (cioos_table_1["depth"] <= 80)]
cioos_e01_95 = cioos_table_1[(cioos_table_1["depth"] >= 90) & (cioos_table_1["depth"] <= 100)]

In [ ]:
cioos_e01_35.columns

In [ ]:
cioos_e01_35.index

In [ ]:
fig, ax = plt.subplots(2, figsize=(15, 8), sharex=True)
   
ax[0].plot(cioos_e01_35.index, cioos_e01_35.sea_water_temperature, linewidth=0.05, c='orange')
ax[0].set_title('Temperature')
   
ax[1].plot(cioos_e01_35.index, cioos_e01_35.sea_water_pratical_salinity, linewidth=0.5, c='blue')
ax[1].set_title("Salinity")
    
    
fig.subplots_adjust(hspace=0.5)
#plt.suptitle("DFO Mooring Station E01 at depth {} metres".format(str(mooring_depth)))
plt.show()

In [ ]:
fig, ax = plt.subplots(2, figsize=(15, 8), sharex=True)
ax[0].plot(cioos_e01_35.index, cioos_e01_35.sea_water_temperature, linewidth=0.05, c='orange')

In [ ]:
#cioos_xarray = cioos_table_1.to_xarray
cioos_e01_35x = cioos_e01_35.to_xarray()
#cioos_e01_75x = cioos_e01_75.to_xarray
#cioos_e01_95x = cioos_e01_95.to_xarray

In [ ]:
cioos_e01_35x

In [ ]:
#confirm the date type is string
type(cioos_e01_35x.time.values[0])

In [ ]:
#cioos_e01_35x['time'] = pd.DatetimeIndex(cioos_e01_35x['time'].values)
cioos_e01_35x['time']  = [pd.Timestamp(ii) for ii in pd.DatetimeIndex(cioos_e01_35x['time'].values)]

In [ ]:
cioos_e01_35x.sea_water_temperature.plot()

In [ ]:
cioos_e01_35x.sea_water_temperature.plot.line()

In [ ]:
cioos_e01_35.TEMPST01.plot()

In [ ]:
fig, ax = plt.subplots()
cb = ax.scatter(cioos_table['time (UTC)'],
           cioos_table['sea_water_temperature (degC)'], 
           c=cioos_table['sea_water_pressure (dbar)'])
fig.colorbar(cb,ax=ax)